# Topic Modeling -- NMF

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import gensim
import time
import time

import nltk
import stanza
import gensim
import re

#topic modeling
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression as lm

# viz
import matplotlib.collections as mcol
from matplotlib.legend_handler import HandlerLineCollection, HandlerTuple
from matplotlib.lines import Line2D
from normalization2 import *
import matplotlib.pyplot as plt
pd.options.display.max_colwidth=500

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

#some ipython magic to show the matplotlib plots inline
%matplotlib inline

In [2]:
#df = pd.read_pickle("/home/zz3hs/git/dspg21oss/data/dspg21oss/readme_lemma_try.pkl")

df = pd.read_csv("/home/zz3hs/git/dspg21oss/data/dspg21oss/tokenized_readmes.csv")

In [3]:
df

,slug,batch,as_of,status,tokens
0,h2oota/emacs-win64-msvc,oss_readme_batch1_1,6/11/21 17:21,Done,"software,foundation,file,license,conditions,di..."
1,Distrotech/mysql-server,oss_readme_batch1_1,6/11/21 17:21,Done,"mysql,server,release,mysql,dual,license,sql,da..."
2,DaichiUeura/Emacs-for-Windows--xj-,oss_readme_batch1_1,6/11/21 17:21,Done,"software,foundation,file,license,conditions,di..."
3,httpgit12/jb4evea-16,oss_readme_batch1_1,6/11/21 17:21,Done,"img,alt,swift,logo,height,src,swift,org,assets..."
4,Tehsurfer/hugo-contrarian,oss_readme_batch1_1,6/11/21 17:21,Done,"contrarian,website,website,arrrbitrage,tehsurf..."
...,...,...,...,...,...
52875,ChrisMichaelPerezSantiago/vuejs-express-api-st...,oss_readme_batch1_9,2021-06-14 15:06:27,Done,"template,starter,vuejs,express,api,starter,ðÿ,..."
52876,jongbinjung/undi,oss_readme_batch1_9,2021-06-14 15:06:27,Done,"readme,md,generated,readme,rmd,edit,file,undi,..."
52877,angelsenra/orphans,oss_readme_batch1_9,2021-06-14 15:06:26,Done,"orphans,collection,projects,gists,abandoned"
52878,polsani/gatsby-starter-netlify-cms,oss_readme_batch1_9,2021-06-14 15:06:26,Done,"note,gatsby,v2,beta,support,check,gatsby,v2,au..."


In [4]:
df["tokens"][1]

'mysql,server,release,mysql,dual,license,sql,database,server,avoidance,doubt,copy,software,released,version,gnu,public,license,mysql,brought,oracle,oracle,affiliates,rights,reserved,license,information,found,copying,file,mysql,foss,license,exception,source,software,applications,licenses,gpl,licensed,mysql,client,libraries,foss,licenses,compatible,version,gnu,public,license,special,exceptions,terms,conditions,gplv2,applied,client,libraries,identified,detail,foss,license,exception,foss,exception,html,legal,licensing,www,mysql,distribution,include,materials,developed,parties,license,attribution,notices,materials,refer,documentation,accompanies,distribution,licenses,party,components,appendix,view,online,documentation,dev,mysql,doc,gplv2,disclaimer,avoidance,doubt,license,choice,gpl,lgpl,apply,oracle,elects,public,license,version,gplv2,time,software,choice,gpl,license,versions,language,indicating,gplv2,version,choice,version,gpl,applied,unspecified,information,mysql,additional,documentation

In [5]:
# function slightly modified from https://nlpforhackers.io/topic-modeling/

def print_topics(topic_term_mat, vectorizer, top_n=10):
    for idx, topic in enumerate(topic_term_mat):  # loop through each row of H.  idx = row index.  topic = actual row
        print("\nTopic %d:" % (idx))
        #print([(vectorizer.get_feature_names()[i], topic[i])  # printing out words corresponding to indices found in next line
                        #for i in topic.argsort()[:-top_n - 1:-1]])  # finding indices of top words in topic
            
        print_list = [(vectorizer.get_feature_names()[i], topic[i])  
                        for i in topic.argsort()[:-top_n - 1:-1]]
        for item in print_list:
            print(item)
            
# Function to format topics as a "list of list of strings".
# Needed for topic coherence function in Gensim

# function modified from https://nlpforhackers.io/topic-modeling/

def str_topics(topic_term_mat, vectorizer, top_n=10):

    #input. top_n: how many words to list per topic.  If -1, then list all words.
       
    topic_words = []
    
    for idx, topic in enumerate(topic_term_mat):  # loop through each row of H.  idx = row index.  topic = actual row
            
        if top_n == -1:   
            topic_words.append([vectorizer.get_feature_names()[i] for i in topic.argsort()[::-1]])
        else:
            topic_words.append([vectorizer.get_feature_names()[i] for i in topic.argsort()[:-top_n - 1:-1]])
    
    str_wds = []
    
    for wds in topic_words:
        str_wds.append(", ".join(wds))    
    
    return str_wds 

In [11]:
# input needed for LDA, NMF (all from Scikit-Learn) is one string per document (not a list of strings)
text = []
abstract_tokens = df["tokens"]

for tokens in abstract_tokens:
    text.append(tokens)

In [13]:
# vectorize the corpus
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=3, lowercase=False)
# calculate the feature matrix
tf_idf = tfidf_vectorizer.fit_transform(text)


In [29]:
shape = tf_idf.shape

print("There are", shape[1], "unique tokens in the corpus of",shape[0], "readmes.")

There are 79212 unique tokens in the corpus of 52880 readmes.


In [19]:
# create model

num_topics = 10

t1 = time.time()
nmf_model = NMF(n_components=num_topics, random_state = 1) #random_state is like a seed in R, control the random number generator
doc_topic = nmf_model.fit_transform(tf_idf)
t2 = time.time()
print(f"  Model run time: {t2-t1}")

topic_term = nmf_model.components_

  Model time: 9.9813973903656


In [20]:
nmf_model

NMF(n_components=10, random_state=1)

In [21]:
topic_frame = pd.DataFrame(doc_topic, columns=["Topic"+" "+str(i) for i in range(num_topics)])
topic_frame

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9
0,0.000000,0.000000,0.023801,0.000000,0.001059,0.000000,0.002217,0.005146,0.000000,0.011032
1,0.000573,0.000566,0.023478,0.000000,0.000142,0.000000,0.012188,0.006180,0.000000,0.000000
2,0.000000,0.000000,0.023567,0.000000,0.001117,0.000000,0.002225,0.005186,0.000000,0.009311
3,0.000000,0.000302,0.006297,0.000449,0.000000,0.000000,0.034107,0.000481,0.000900,0.013354
4,0.002334,0.007874,0.000000,0.103505,0.000000,0.003799,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
52875,0.000413,0.001398,0.002780,0.008090,0.028976,0.016908,0.013175,0.001279,0.027690,0.000000
52876,0.000000,0.000000,0.005827,0.000000,0.000000,0.000000,0.029985,0.000000,0.000000,0.015631
52877,0.000000,0.000000,0.001450,0.000138,0.000557,0.000000,0.002175,0.000000,0.000000,0.000000
52878,0.000043,0.000108,0.000000,0.162910,0.076369,0.000000,0.000000,0.000110,0.018272,0.001385


In [22]:
topic_words = str_topics(topic_term, tfidf_vectorizer, top_n=10)
topic_words

['academic, sourcethemes, docs, kickstart, hugo, gcushen, cushen, computer, twitter, rstudio',
 'wowchemy, hugo, academic, twitter, admin, resumé, sponsorship, template, view, docs',
 'file, data, python, user, api, id, docker, code, py, server',
 'cms, netlify, deploy, gatsby, templates, gyp, npm, media, uploadcare, cloudinary',
 'gatsby, starter, gatsbyjs, blog, theme, site, js, www, default, file',
 'writeup, traffic, udacity, sign, carnd, project, classifier, notebook, model, images',
 'io, svg, travis, org, ci, img, badge, license, shields, status',
 'moodle, learning, partners, org, platform, world, moodlecloud, personalised, educators, learners',
 'npm, run, react, js, angular, app, server, node, build, start',
 'cmake, project, kalman, simulator, ide, cpp, rmse, uwebsocketio, filter, linux']